In [ ]:

import requests as re
import pandas as pd
url = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"
d = re.get(url).json()


tk_df = pd.DataFrame.from_dict(d)
tk_df['expiry'] = pd.to_datetime(tk_df['expiry'])
dfx = tk_df.astype({'strike': float})
tk_df.to_csv("d.csv")


# Get  token for nifty 50 spot idx 

df_filtered = dfx[(dfx['symbol'] == 'Nifty 50') &
                   (dfx['instrumenttype'] == 'AMXIDX') &
                   (dfx['exch_seg'] == 'NSE') &
                   (dfx['name'] == 'NIFTY')]
print(str(df_filtered.token.iloc[0]))

# Get  token for nifty 50 idx  put premium 
def rPutToken(range):
    df_filtered = dfx[(dfx['symbol'] == 'Nifty 50') &
                    (dfx['instrumenttype'] == 'OPTIDX') &
                    (dfx['exch_seg'] == 'NSE') &
                    (dfx['name'] == 'NIFTY')]
    print(str(df_filtered.token.iloc[0]))
    list_of_token = []
    return list_of_token
# Get  token for nifty 50 idx   call premium 
def rCallToken(range):
    df_filtered = dfx[(dfx['symbol'] == 'Nifty 50') &
                    (dfx['instrumenttype'] == 'OPTIDX') &
                    (dfx['exch_seg'] == 'NSE') &
                    (dfx['name'] == 'NIFTY')]
    print(str(df_filtered.token.iloc[0]))
    list_of_token = []
    return list_of_token



# BotLogic

# BotRenderLogic&  BotSaveLogic


In [ ]:
from lightweight_charts import Chart


Chart = Chart()

Chart.update_from_tick()


In [ ]:
!date -d @ 

In [ ]:
!date -d "2 days ago"


In [ ]:
!date   -d  "now"


In [ ]:
!date -u  -d  "now"



In [4]:
# dict of str, list of dict etc 
dict_obj = {"topic":"publicTrade.BTCUSDT","type":"snapshot","ts":1725607296272,"data":[{"T":1725607296271,"s":"BTCUSDT","S":"Buy","v":"0.007","p":"55761.90","L":"ZeroPlusTick","i":"1d42b529-37ca-5161-b72b-3b349e93180e","BT":"false"}]}


In [ ]:
# dict of list 
data_dict = dict_obj['data']
data_dict

In [ ]:
tick_list = data_dict[0]
tick_list

In [ ]:
tick_list['T']

In [ ]:
! date -d @1725607296.271

In [ ]:
! date -d @1725607296.333

In [3]:
import pandas as pd
import json 
from datetime import datetime, timedelta

class ChartData:
    def __init__(self):
        self._last_bar = None
        self._current_bar = None
        self._bar_start_time = None

    def _series_datetime_format(self, series: pd.Series):
        series['time'] = pd.to_datetime(series['time'])
        return series

    def _round_to_nearest_5_seconds(self, time: datetime):
        # Round down to the nearest 5 seconds
        seconds = (time.second // 5) * 5
        return time.replace(second=seconds, microsecond=0)

    def update_from_tick(self, series: pd.Series):
        """Updates the data from a tick."""
        series = self._series_datetime_format(series)
        current_time = series['time']

        # Round current time to the nearest 5-second boundary
        rounded_time = self._round_to_nearest_5_seconds(current_time)

        # Create a new bar every 5 seconds
        if self._bar_start_time is None or rounded_time >= self._bar_start_time + timedelta(seconds=5):
            self._bar_start_time = rounded_time  # Use the rounded time for candlesticks
            self._current_bar = pd.Series({
                'time': self._bar_start_time.timestamp(),
                'open': series['price'],
                'high': series['price'],
                'low': series['price'],
                'close': series['price']
            })
            if self._last_bar is not None:
                yield self._last_bar.to_dict()
            self._last_bar = self._current_bar
        else:
            # Update current bar
            self._current_bar['high'] = max(self._current_bar['high'], series['price'])
            self._current_bar['low'] = min(self._current_bar['low'], series['price'])
            self._current_bar['close'] = series['price']

        yield self._current_bar.to_dict()


chart_data = ChartData()

def on_open(ws):
    print("Bybit WebSocket Opened")
    ws.send('{"op": "subscribe","args": ["publicTrade.BTCUSDT"]}')

def on_error(ws, error):
    print(f"WebSocket Error: {error}")

def on_message(ws, message):
    tick = json.loads(message)
    if tick.get('topic') == 'publicTrade.BTCUSDT':
        trade_data = tick['data'][0]
        tick_time = pd.to_datetime(trade_data['T'] / 1000, unit='s')
        tick_data = {'time': tick_time, 'price': float(trade_data['p'])}
        for updated_bar in chart_data.update_from_tick(pd.Series(tick_data)):
            print('update_chart', updated_bar)


ws_url = "wss://stream.bybit.com/v5/public/linear"

import websocket

ws = websocket.WebSocketApp(ws_url,
                            on_message=on_message,
                            on_error=on_error,         
                            on_open=on_open)

from threading import Thread
wst = Thread(target=ws.run_forever)
wst.daemon = True
wst.start()


Bybit WebSocket Opened
update_chart {'time': 1725609955.000001, 'open': 55720.0, 'high': 55720.0, 'low': 55720.0, 'close': 55720.0}
update_chart {'time': 1725609955.000001, 'open': 55720.0, 'high': 55720.0, 'low': 55720.0, 'close': 55720.0}
update_chart {'time': 1725609955.000001, 'open': 55720.0, 'high': 55720.1, 'low': 55720.0, 'close': 55720.1}
update_chart {'time': 1725609955.000001, 'open': 55720.0, 'high': 55720.1, 'low': 55720.0, 'close': 55720.1}
update_chart {'time': 1725609955.000001, 'open': 55720.0, 'high': 55720.1, 'low': 55720.0, 'close': 55720.1}
update_chart {'time': 1725609955.000001, 'open': 55720.0, 'high': 55720.1, 'low': 55720.0, 'close': 55720.1}
update_chart {'time': 1725609955.000001, 'open': 55720.0, 'high': 55720.1, 'low': 55720.0, 'close': 55720.1}
update_chart {'time': 1725609955.000001, 'open': 55720.0, 'high': 55720.1, 'low': 55720.0, 'close': 55720.1}
update_chart {'time': 1725609955.000001, 'open': 55720.0, 'high': 55720.1, 'low': 55720.0, 'close': 55720

In [1]:
!date -d @1725609955

Friday 06 September 2024 01:35:55 PM IST


In [2]:
!date -d @1725609960

Friday 06 September 2024 01:36:00 PM IST


In [3]:
!date -d @1725609965

Friday 06 September 2024 01:36:05 PM IST


In [ ]:
Bybit WebSocket Opened
update_chart {'time': 1725609955.000001, 'open': 55720.0, 'high': 55720.0, 'low': 55720.0, 'close': 55720.0}
update_chart {'time': 1725609955.000001, 'open': 55720.0, 'high': 55720.0, 'low': 55720.0, 'close': 55720.0}
update_chart {'time': 1725609955.000001, 'open': 55720.0, 'high': 55720.1, 'low': 55720.0, 'close': 55720.1}
update_chart {'time': 1725609955.000001, 'open': 55720.0, 'high': 55720.1, 'low': 55720.0, 'close': 55720.1}
update_chart {'time': 1725609955.000001, 'open': 55720.0, 'high': 55720.1, 'low': 55720.0, 'close': 55720.1}
update_chart {'time': 1725609955.000001, 'open': 55720.0, 'high': 55720.1, 'low': 55720.0, 'close': 55720.1}
update_chart {'time': 1725609955.000001, 'open': 55720.0, 'high': 55720.1, 'low': 55720.0, 'close': 55720.1}
update_chart {'time': 1725609955.000001, 'open': 55720.0, 'high': 55720.1, 'low': 55720.0, 'close': 55720.1}
update_chart {'time': 1725609955.000001, 'open': 55720.0, 'high': 55720.1, 'low': 55720.0, 'close': 55720.1}
update_chart {'time': 1725609955.000001, 'open': 55720.0, 'high': 55720.1, 'low': 55720.0, 'close': 55720.1}
update_chart {'time': 1725609955.000001, 'open': 55720.0, 'high': 55720.7, 'low': 55720.0, 'close': 55720.7}
update_chart {'time': 1725609955.000001, 'open': 55720.0, 'high': 55721.2, 'low': 55720.0, 'close': 55721.2}
update_chart {'time': 1725609955.000001, 'open': 55720.0, 'high': 55724.5, 'low': 55720.0, 'close': 55724.5}
update_chart {'time': 1725609955.000001, 'open': 55720.0, 'high': 55724.5, 'low': 55720.0, 'close': 55724.5}
update_chart {'time': 1725609955.000001, 'open': 55720.0, 'high': 55724.5, 'low': 55720.0, 'close': 55724.5}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.5, 'low': 55724.5, 'close': 55724.5}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.6, 'low': 55724.5, 'close': 55724.6}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.6, 'low': 55724.5, 'close': 55724.5}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.6, 'low': 55724.5, 'close': 55724.5}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.6, 'low': 55724.5, 'close': 55724.5}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.6, 'low': 55724.5, 'close': 55724.5}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.6, 'low': 55724.5, 'close': 55724.5}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.6, 'low': 55722.1, 'close': 55722.1}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.6, 'low': 55722.1, 'close': 55722.1}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.6, 'low': 55722.1, 'close': 55722.1}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.6, 'low': 55719.9, 'close': 55719.9}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.6, 'low': 55719.9, 'close': 55720.0}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.6, 'low': 55719.9, 'close': 55720.0}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.6, 'low': 55719.9, 'close': 55720.0}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.6, 'low': 55719.9, 'close': 55720.0}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.6, 'low': 55719.9, 'close': 55720.0}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.6, 'low': 55719.9, 'close': 55720.0}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.6, 'low': 55719.9, 'close': 55720.0}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.6, 'low': 55719.9, 'close': 55720.0}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.6, 'low': 55719.9, 'close': 55720.0}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.6, 'low': 55719.9, 'close': 55722.1}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.6, 'low': 55719.9, 'close': 55722.0}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.6, 'low': 55719.9, 'close': 55722.1}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.6, 'low': 55719.9, 'close': 55722.0}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.6, 'low': 55719.9, 'close': 55722.1}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.6, 'low': 55719.9, 'close': 55722.0}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.6, 'low': 55719.9, 'close': 55722.0}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.6, 'low': 55719.9, 'close': 55722.0}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.6, 'low': 55719.9, 'close': 55722.0}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.6, 'low': 55719.9, 'close': 55722.0}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.6, 'low': 55719.9, 'close': 55722.0}
update_chart {'time': 1725609960.000001, 'open': 55724.5, 'high': 55724.6, 'low': 55719.9, 'close': 55722.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55722.0, 'low': 55722.0, 'close': 55722.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55722.0, 'low': 55722.0, 'close': 55722.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55722.0, 'low': 55722.0, 'close': 55722.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55722.0, 'low': 55722.0, 'close': 55722.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55722.0, 'low': 55722.0, 'close': 55722.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55722.0, 'low': 55722.0, 'close': 55722.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55722.1, 'low': 55722.0, 'close': 55722.1}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55722.1, 'low': 55722.0, 'close': 55722.1}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55722.1, 'low': 55722.0, 'close': 55722.1}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55722.1, 'low': 55722.0, 'close': 55722.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55722.1, 'low': 55722.0, 'close': 55722.1}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55722.1, 'low': 55722.0, 'close': 55722.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55722.1, 'low': 55722.0, 'close': 55722.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55722.1, 'low': 55722.0, 'close': 55722.1}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55722.1, 'low': 55722.0, 'close': 55722.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55722.1, 'low': 55722.0, 'close': 55722.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55722.1, 'low': 55722.0, 'close': 55722.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55722.1, 'low': 55722.0, 'close': 55722.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55722.1, 'low': 55722.0, 'close': 55722.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55722.1, 'low': 55722.0, 'close': 55722.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55722.1, 'low': 55722.0, 'close': 55722.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55722.1, 'low': 55722.0, 'close': 55722.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55722.1, 'low': 55722.0, 'close': 55722.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55722.1, 'low': 55722.0, 'close': 55722.1}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55722.1, 'low': 55722.0, 'close': 55722.1}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55722.0, 'close': 55723.3}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55722.0, 'close': 55723.3}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55722.0, 'close': 55723.2}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55718.5, 'close': 55718.5}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55718.5, 'close': 55718.5}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55718.5, 'close': 55718.5}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55718.5, 'close': 55718.5}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55718.5, 'close': 55718.5}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55718.5, 'close': 55718.5}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55718.5, 'close': 55718.5}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55717.0, 'close': 55717.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55711.2, 'close': 55711.2}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55711.2, 'close': 55711.2}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55711.0, 'close': 55711.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55711.0, 'close': 55711.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55711.0, 'close': 55711.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55710.9, 'close': 55710.9}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55710.9, 'close': 55710.9}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55710.9, 'close': 55711.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55710.9, 'close': 55711.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55710.9, 'close': 55711.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55710.9, 'close': 55711.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55710.9, 'close': 55711.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55710.9, 'close': 55711.3}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55710.9, 'close': 55711.3}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55710.9, 'close': 55711.3}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55710.9, 'close': 55711.9}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55710.9, 'close': 55713.2}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55710.9, 'close': 55714.5}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55710.9, 'close': 55714.5}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55710.9, 'close': 55714.5}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55710.9, 'close': 55714.5}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55710.9, 'close': 55715.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55710.9, 'close': 55715.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55710.9, 'close': 55715.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55710.9, 'close': 55715.0}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55710.9, 'close': 55715.1}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55710.9, 'close': 55715.8}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55710.9, 'close': 55719.3}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55710.9, 'close': 55719.3}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55710.9, 'close': 55719.3}
update_chart {'time': 1725609965.000001, 'open': 55722.0, 'high': 55723.3, 'low': 55710.9, 'close': 55719.3}
update_chart {'time': 1725609970.000001, 'open': 55719.3, 'high': 55719.3, 'low': 55719.3, 'close': 55719.3}
update_chart {'time': 1725609970.000001, 'open': 55719.3, 'high': 55719.3, 'low': 55719.3, 'close': 55719.3}
update_chart {'time': 1725609970.000001, 'open': 55719.3, 'high': 55719.4, 'low': 55719.3, 'close': 55719.4}
update_chart {'time': 1725609970.000001, 'open': 55719.3, 'high': 55719.4, 'low': 55719.3, 'close': 55719.3}
update_chart {'time': 1725609970.000001, 'open': 55719.3, 'high': 55719.4, 'low': 55719.3, 'close': 55719.3}
update_chart {'time': 1725609970.000001, 'open': 55719.3, 'high': 55719.4, 'low': 55719.3, 'close': 55719.4}
update_chart {'time': 1725609970.000001, 'open': 55719.3, 'high': 55719.4, 'low': 55719.3, 'close': 55719.4}
update_chart {'time': 1725609970.000001, 'open': 55719.3, 'high': 55722.1, 'low': 55719.3, 'close': 55722.1}
update_chart {'time': 1725609970.000001, 'open': 55719.3, 'high': 55723.2, 'low': 55719.3, 'close': 55723.2}
update_chart {'time': 1725609970.000001, 'open': 55719.3, 'high': 55723.2, 'low': 55719.3, 'close': 55722.1}
update_chart {'time': 1725609970.000001, 'open': 55719.3, 'high': 55723.2, 'low': 55719.3, 'close': 55722.1}
update_chart {'time': 1725609970.000001, 'open': 55719.3, 'high': 55723.2, 'low': 55719.3, 'close': 55722.0}
update_chart {'time': 1725609970.000001, 'open': 55719.3, 'high': 55723.2, 'low': 55719.3, 'close': 55722.0}
update_chart {'time': 1725609970.000001, 'open': 55719.3, 'high': 55723.2, 'low': 55719.3, 'close': 55722.0}
update_chart {'time': 1725609970.000001, 'open': 55719.3, 'high': 55723.2, 'low': 55719.3, 'close': 55722.1}
update_chart {'time': 1725609970.000001, 'open': 55719.3, 'high': 55723.2, 'low': 55719.3, 'close': 55722.0}
update_chart {'time': 1725609970.000001, 'open': 55719.3, 'high': 55723.2, 'low': 55719.3, 'close': 55722.0}
update_chart {'time': 1725609970.000001, 'open': 55719.3, 'high': 55723.2, 'low': 55719.3, 'close': 55722.0}
update_chart {'time': 1725609970.000001, 'open': 55719.3, 'high': 55723.2, 'low': 55719.3, 'close': 55722.0}
update_chart {'time': 1725609970.000001, 'open': 55719.3, 'high': 55723.2, 'low': 55719.3, 'close': 55722.0}
update_chart {'time': 1725609970.000001, 'open': 55719.3, 'high': 55723.2, 'low': 55719.3, 'close': 55722.1}
update_chart {'time': 1725609970.000001, 'open': 55719.3, 'high': 55723.2, 'low': 55719.3, 'close': 55722.1}
update_chart {'time': 1725609970.000001, 'open': 55719.3, 'high': 55723.2, 'low': 55719.3, 'close': 55722.0}
update_chart {'time': 1725609970.000001, 'open': 55719.3, 'high': 55723.2, 'low': 55719.3, 'close': 55722.0}
update_chart {'time': 1725609970.000001, 'open': 55719.3, 'high': 55723.2, 'low': 55719.3, 'close': 55722.0}
update_chart {'time': 1725609970.000001, 'open': 55719.3, 'high': 55723.2, 'low': 55719.3, 'close': 55722.0}
update_chart {'time': 1725609970.000001, 'open': 55719.3, 'high': 55723.2, 'low': 55719.3, 'close': 55722.0}
update_chart {'time': 1725609970.000001, 'open': 55719.3, 'high': 55723.2, 'low': 55719.3, 'close': 55722.0}
update_chart {'time': 1725609970.000001, 'open': 55719.3, 'high': 55723.2, 'low': 55719.3, 'close': 55722.0}
update_chart {'time': 1725609975.000001, 'open': 55722.1, 'high': 55722.1, 'low': 55722.1, 'close': 55722.1}
update_chart {'time': 1725609975.000001, 'open': 55722.1, 'high': 55722.1, 'low': 55722.1, 'close': 55722.1}
update_chart {'time': 1725609975.000001, 'open': 55722.1, 'high': 55722.1, 'low': 55722.0, 'close': 55722.0}
update_chart {'time': 1725609975.000001, 'open': 55722.1, 'high': 55722.1, 'low': 55722.0, 'close': 55722.0}
update_chart {'time': 1725609975.000001, 'open': 55722.1, 'high': 55722.1, 'low': 55722.0, 'close': 55722.0}
update_chart {'time': 1725609975.000001, 'open': 55722.1, 'high': 55722.1, 'low': 55722.0, 'close': 55722.1}
update_chart {'time': 1725609975.000001, 'open': 55722.1, 'high': 55722.1, 'low': 55722.0, 'close': 55722.0}
update_chart {'time': 1725609975.000001, 'open': 55722.1, 'high': 55722.1, 'low': 55722.0, 'close': 55722.0}
update_chart {'time': 1725609975.000001, 'open': 55722.1, 'high': 55722.1, 'low': 55722.0, 'close': 55722.0}
update_chart {'time': 1725609975.000001, 'open': 55722.1, 'high': 55722.1, 'low': 55720.9, 'close': 55720.9}
update_chart {'time': 1725609975.000001, 'open': 55722.1, 'high': 55722.1, 'low': 55719.3, 'close': 55719.3}
update_chart {'time': 1725609975.000001, 'open': 55722.1, 'high': 55722.1, 'low': 55719.3, 'close': 55719.3}
update_chart {'time': 1725609975.000001, 'open': 55722.1, 'high': 55722.1, 'low': 55719.2, 'close': 55719.2}
update_chart {'time': 1725609975.000001, 'open': 55722.1, 'high': 55722.1, 'low': 55718.8, 'close': 55718.8}
update_chart {'time': 1725609975.000001, 'open': 55722.1, 'high': 55722.1, 'low': 55718.7, 'close': 55718.7}
